In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import yelp_util

In [2]:
yelp_review = pd.read_pickle('data/yelp_academic_dataset_review.pickle') # read yelp review pickle

In [3]:
review_list = list(yelp_review.text.iloc[10000:11000]) # example of review
word2vec_model = yelp_util.create_word2vec_model(review_list, size=100) # create word2vec model

breaking into sentence...
trianing word2vec model...


In [4]:
review_words_stream = yelp_util.get_stream_seq(review_list, word2vec_model)

In [5]:
embeddings = yelp_util.get_word_embedding(word2vec_model) # get word vector embedding

Vocabulary size:  1294
Word vector dimension:  100


In [11]:
args = yelp_util.InputParameter()
# correct default parameters
args.vocab_size = embeddings.shape[0]
args.rnn_size = embeddings.shape[1]
args.batch_size = 10
args.seq_length = 25

In [7]:
model = yelp_util.ReviewModel(args)

In [13]:
batch_size, seq_length = 10, 25
sequence_streamer = yelp_util.SeqStream(review_words_stream, batch_size, seq_length) # where review words stream is the stream of word2vec index

In [16]:
print len(sequence_streamer.tensor) # class of reviews stream

103250


In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    saver = tf.train.Saver(tf.all_variables())
    for e in xrange(args.num_epochs):
        sess.run(tf.assign(model.lr, args.learning_rate * (args.decay_rate ** e)))
        sequence_streamer.reset_batch_pointer()
        state = model.initial_state.eval(session=sess)
        for b in xrange(sequence_streamer.num_batches):
            start = time.time()
            x, y = sequence_streamer.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state: state, model.embedding: embeddings}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
            if ((e * sequence_streamer.num_batches + b) % 500) == 0:
                print "{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                    .format(e * sequence_streamer.num_batches + b,
                            args.num_epochs * sequence_streamer.num_batches,
                            e, train_loss, end - start)

0/20650 (epoch 0), train_loss = 7.710, time/batch = 0.591
500/20650 (epoch 1), train_loss = 5.608, time/batch = 0.126
1000/20650 (epoch 2), train_loss = 5.425, time/batch = 0.147
1500/20650 (epoch 3), train_loss = 5.172, time/batch = 0.138
2000/20650 (epoch 4), train_loss = 5.175, time/batch = 0.125
2500/20650 (epoch 6), train_loss = 4.999, time/batch = 0.146
3000/20650 (epoch 7), train_loss = 4.829, time/batch = 0.126
3500/20650 (epoch 8), train_loss = 4.774, time/batch = 0.133
4000/20650 (epoch 9), train_loss = 5.017, time/batch = 0.144